# 06 Entrenamiento y Validación del Modelo (Google Colab)
*Cuaderno para ejecutar en Google Colab*


## Justificación del Entrenamiento en Google Colab

Debido a las **limitaciones de recursos de hardware disponibles localmente**, particularmente la **ausencia de una Unidad de Procesamiento Gráfico (GPU)**, el entrenamiento del modelo de detección de objetos YOLOv8n-obb se realizará en la plataforma **Google Colab**. Colab proporciona acceso gratuito a GPUs potentes, como la Tesla T4, que son **esenciales para acelerar significativamente el proceso de entrenamiento** de modelos de aprendizaje profundo. Entrenar un modelo complejo como YOLOv8 en una CPU local llevaría tiempos prohibitivamente largos, haciendo inviable la experimentación y el ajuste eficiente del modelo. Por lo tanto, el uso de la infraestructura de Google Colab **optimiza el tiempo de entrenamiento y permite aprovechar la potencia de las GPUs** para lograr resultados en un plazo razonable.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Navegar a tu proyecto
%cd /content/drive/My\ Drive/TFM_BALSAS

Mounted at /content/drive
/content/drive/My Drive/TFM_BALSAS


## GPU

Nos aseguramos que tenemos acceso a GPU. Usamos `nvidia-smi` para hacerlo.

In [3]:
!nvidia-smi

Sat Apr  5 12:42:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import os
HOME = os.getcwd()
print(HOME)

/content/drive/MyDrive/TFM_BALSAS


## Importar librerias necesarias

In [5]:
# Pip install method (recommended)
!pip install ultralytics --upgrade -q

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 40.9/112.6 GB disk)


In [6]:
from ultralytics import YOLO

from IPython.display import display, Image

## Entrenamiento personalizado

In [11]:
# Cargar el modelo preentrenado
model = YOLO('/content/drive/My Drive/TFM_BALSAS/yolov8n-obb.pt')

In [12]:
%%time
# Entrenar el modelo
results = model.train(
    data="dataset/data.yaml",    # Ruta al archivo YAML
    epochs=100,                  # Aumentado para mejor convergencia
    patience=15,                 # Mayor paciencia para evitar paradas prematuras
    imgsz=640,                   # Tamaño estándar para YOLOv8
    batch=8,                     # Reducido para evitar problemas de memoria en T4
    optimizer="AdamW",           # Optimizer más efectivo para detección de objetos
    lr0=0.01,                    # Learning rate inicial
    lrf=0.001,                   # Learning rate final
    momentum=0.937,              # Momentum para el optimizador
    weight_decay=0.0005,         # Weight decay para reducir overfitting
    warmup_epochs=3,             # Epochs de calentamiento
    warmup_momentum=0.8,         # Momentum durante el calentamiento
    warmup_bias_lr=0.1,          # Learning rate para bias durante calentamiento
    box=7.5,                     # Peso para pérdida de bounding box
    cls=0.5,                     # Peso para pérdida de clasificación
    dfl=1.5,                     # Peso para pérdida de distribución
    plots=True,                  # Generar gráficos
    save=True,                   # Guardar checkpoints
    cache=False,                 # No usar cache (para evitar problemas de memoria)
    device=0,                    # Usar GPU
    workers=4,                   # Aumentado ligeramente para mejor paralelismo
    project="deteccion_balsas",  # Nombre del proyecto
    name="yolov8n_obb_run1",     # Nombre del experimento
    exist_ok=True                # Continuar entrenamientos interrumpidos
)

Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=obb, mode=train, model=/content/drive/My Drive/TFM_BALSAS/yolov8n-obb.pt, data=dataset/data.yaml, epochs=100, time=None, patience=15, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=4, project=deteccion_balsas, name=yolov8n_obb_run1, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fals

100%|██████████| 755k/755k [00:00<00:00, 18.3MB/s]


Overriding model.yaml nc=15 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/My Drive/TFM_BALSAS/dataset/labels/train... 581 images, 0 backgrounds, 0 corrupt: 100%|██████████| 581/581 [04:29<00:00,  2.15it/s]

train: WARNING ⚠️ /content/drive/My Drive/TFM_BALSAS/dataset/images/train/88.tif: 1 duplicate labels removed


...

    100/100      3.04G     0.9625     0.6364      1.793          5        640: 100%|██████████| 73/73 [00:21<00:00,  3.47it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:06<00:00,  3.48it/s]

                   all        166        224      0.892      0.866      0.893      0.575



100 epochs completed in 0.786 hours.
Optimizer stripped from deteccion_balsas/yolov8n_obb_run1/weights/last.pt, 6.5MB
Optimizer stripped from deteccion_balsas/yolov8n_obb_run1/weights/best.pt, 6.5MB

Validating deteccion_balsas/yolov8n_obb_run1/weights/best.pt...
Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-obb summary (fused): 81 layers, 3,077,414 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:06<00:00,  3.34it/s]


                   all        166        224      0.883      0.879      0.897      0.574
Speed: 0.4ms preprocess, 6.5ms inference, 0.0ms loss, 9.8ms postprocess per image
Results saved to deteccion_balsas/yolov8n_obb_run1
CPU times: user 22min 24s, sys: 1min 19s, total: 23min 44s
Wall time: 53min 43s


## Evaluación del modelo con el Conjunto de Validación (val)  

El conjunto de validación se utiliza típicamente durante el entrenamiento para monitorear el rendimiento del modelo. Puedes evaluar el modelo en este conjunto usando el método model.val().

In [13]:
# Evaluar el modelo en el conjunto de validación
results_val = model.val(plots=True)   # Evalúa en el conjunto de test configurado en data.yaml

# Imprimir métricas clave
print("Resultados en el conjunto de validación:")
print(f"mAP@0.5: {results_val.box.map50}")  # mAP con IoU=0.5
print(f"mAP@0.5:0.95: {results_val.box.map}")  # mAP con IoU entre 0.5 y 0.95

Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-obb summary (fused): 81 layers, 3,077,414 parameters, 0 gradients, 8.3 GFLOPs


val: Scanning /content/drive/My Drive/TFM_BALSAS/dataset/labels/val.cache... 166 images, 0 backgrounds, 0 corrupt: 100%|██████████| 166/166 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/drive/My Drive/TFM_BALSAS/dataset/images/val/259.tif: 1 duplicate labels removed
val: WARNING ⚠️ /content/drive/My Drive/TFM_BALSAS/dataset/images/val/594.tif: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:06<00:00,  3.12it/s]


                   all        166        224      0.883      0.879      0.897      0.574
Speed: 2.7ms preprocess, 8.9ms inference, 0.0ms loss, 7.1ms postprocess per image
Results saved to deteccion_balsas/yolov8n_obb_run1
Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


val: Scanning /content/drive/My Drive/TFM_BALSAS/dataset/labels/val.cache... 166 images, 0 backgrounds, 0 corrupt: 100%|██████████| 166/166 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/drive/My Drive/TFM_BALSAS/dataset/images/val/259.tif: 1 duplicate labels removed
val: WARNING ⚠️ /content/drive/My Drive/TFM_BALSAS/dataset/images/val/594.tif: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:06<00:00,  3.11it/s]


                   all        166        224      0.883      0.879      0.897      0.574
Speed: 3.8ms preprocess, 7.8ms inference, 0.0ms loss, 8.6ms postprocess per image
Results saved to deteccion_balsas/yolov8n_obb_run1
Resultados en el conjunto de validación:
mAP@0.5: 0.8972523797677318
mAP@0.5:0.95: 0.5738130941727748


## Valoración de los resultados

Los resultadosp roporcionados por la evaluación muestran un desempeño muy bueno del modelo **YOLOv8n-obb** en el conjunto de validación. REsumimos a continuación los puntos más destacados:

### Métricas clave
Las métricas principales son:

- **mAP@0.5 (mean Average Precision a IoU=0.5):**  
  - Valor: **0.897** (~89.7%)  
  - Significado: Indica la precisión promedio del modelo cuando se considera que una predicción es correcta si su IoU (Intersection over Union) con la caja real es al menos del 50%.  
  - Evaluación: Este valor es excelente. Un mAP@0.5 superior al 85% sugiere que el modelo está detectando objetos con alta precisión.

- **mAP@0.5:0.95 (mean Average Precision a IoU entre 0.5 y 0.95):**  
  - Valor: **0.574** (~57.4%)  
  - Significado: Es una métrica más estricta que evalúa la precisión promedio en diferentes umbrales de IoU (desde 0.5 hasta 0.95).  
  - Evaluación: Este valor también es sólido. Un mAP@0.5:0.95 superior al 50% indica que el modelo no solo detecta bien las cajas, sino que también localiza los objetos con precisión.

- **Precisión (P):**  
  - Valor: **0.883** (~88.3%)  
  - Significado: Proporción de predicciones correctas entre todas las predicciones realizadas.  
  - Evaluación: Una precisión del 88.3% es muy buena, lo que indica que el modelo tiene pocas falsas alarmas.

- **Recall (R):**  
  - Valor: **0.879** (~87.9%)  
  - Significado: Proporción de objetos reales detectados correctamente.  
  - Evaluación: Un recall del 87.9% es excelente, lo que sugiere que el modelo detecta la mayoría de los objetos presentes en las imágenes.

### Conclusión
- **Desempeño:**  
  El modelo ha alcanzado métricas muy buenas en el conjunto de validación, con un mAP@0.5 del **89.7%** y un mAP@0.5:0.95 del **57.4%**. Estos valores indican que el modelo es capaz de detectar y localizar objetos con alta precisión.

- **Velocidad:**  
  La inferencia es rápida (~20 ms por imagen).

En resumen, los resultados obtenidos son **muy buenos** y demuestran que el modelo YOLOv8n-obb está funcionando muy bien para la tarea de detección de balsas-obb. Con una limpieza adicional de los datos y posibles ajustes en el entrenamiento se podrías alcanzar métricas mejores. 